Import necessary packages / functions

In [55]:
import tarfile
import numpy as np
import pandas as pd
import random
import math
import time
from sklearn import preprocessing

Declare / implement necessary functions

In [56]:
def randBinStr(listLen):
    cds = []
    for i in range(0, listLen):
        cds.append(np.round(random.uniform(0,1)))
    cds = np.array(cds)
    return cds

def cosAngle(v1, v2):
    return (np.arccos(round(np.sum(v1 * v2),5)) / np.pi) * 180

def cosDist(v1, v2):
    return 1 - np.sum(v1 * v2)

Set up the parameters

In [57]:
fields = ['duration','end_of_fade_in','key', 'loudness','mode','start_of_fade_out','tempo','time_signature']
r = int(64)
b = int(3)
epsilon = int(2)

Read and prepare data for duplicate detection

In [58]:
tar = tarfile.open('millionsongsubset_full.tar.gz', 'r')
members = tar.getmembers()

summary = pd.HDFStore(members[5].name)
totalFields = len(fields)
N = 10000

F = []

for f in fields:
    F.append(summary['/analysis/songs'][f])

X = []

for i in range(0, N):
    x = []
    for f in F:
        x.append(f[i])
    X.append(x)
X = np.array(X)

X = preprocessing.scale(X)
for i in range(0, N):
    X[i,:] = X[i,:] / np.sqrt(sum(X[i,:] * X[i,:]))

Start timing the LSH Cos similarity code

In [59]:
execBegin = time.clock()

Generate hash functions

In [60]:
totalHash = r * b
checkedVal = []
hashFunctions = []
sumCheck = [pow(2,x) for x in range(totalFields-1, -1, -1)]
i = 0
while(i < totalHash):
    randHash = randBinStr(totalFields)
    hashval = sum(sumCheck * randHash)
    
    if(hashval not in checkedVal):
        checkedVal.append(hashval)
        hashFunctions.append(randHash * 2 - 1)
        i += 1

Generate signature matrix

In [61]:
X_signature = []
for i in range(0, totalHash):
    x_s = []
    Hf = hashFunctions[i]
    for x in X:
        x_s.append((np.sum(Hf * x) > 0) * 1)
    X_signature.append(x_s)
X_signature = np.array(X_signature)

Assign appropriate bucket to songs

In [62]:
X_pairs = []
decConvVect = [2**x for x in range(0,r)]
for b_i in range(0, b):
    bins = []
    for x_i in range(0, N):
        decVal = sum(X_signature[b_i * r : (b_i + 1) * r, x_i] * decConvVect)
        bins.append([x_i, decVal])
    X_pairs.append(bins)
X_pairs = np.array(X_pairs)

Generate candidate pairs

In [63]:
c_pairs = []
for b_i in range(0,b):
    X_s = sorted(X_pairs[b_i], key = lambda i: i[1])
    for ixs in range(0, N - 1):
        if(X_s[ixs][1] == X_s[ixs + 1][1]):
            if([X_s[ixs][0], X_s[ixs + 1][0]] not in c_pairs):
                c_pairs.append([X_s[ixs][0], X_s[ixs + 1][0]])

Compute duplicates from candidates

In [64]:
pairs = []
for cp in c_pairs:
    if(cosAngle(X[int(cp[0])], X[int(cp[1])]) < epsilon):
        pairs.append(cp)

End timing the LSH Cos similarity code

In [65]:
execTime = time.clock() - execBegin

Print results

In [66]:
print("The LSH code took ", execTime, "ms to detect duplicates\n")
print("Total candidate pairs found: ", len(c_pairs),"\n")
print("Total duplicate entries found: ", len(pairs),"\n")

print("Indexes of duplicate pairs: \n")
for p in pairs:
    print(int(p[0]), int(p[1]))

The LSH code took  11.306169899786255 ms to detect duplicates

Total candidate pairs found:  4514 

Total duplicate entries found:  16 

Indexes of duplicate pairs: 

2597 2785
1063 8948
497 582
6983 8115
612 2569
585 2035
8449 8543
70 6427
908 3742
1256 9811
1899 8632
5970 8978
4826 5442
437 2747
2361 8286
4076 8632


Run the brute force implementation for time comparison

In [53]:
beginBF = time.clock()
bruteForcePairs = []
for i in range(0, N):
    for j in range(i + 1, N):
        if(cosAngle(X[i], X[j]) < epsilon):
            bruteForcePairs.append([i,j])
BFtime = time.clock() - beginBF

In [70]:
print("The brute force code took ", BFtime, "ms to detect duplicates\n")
print("Total duplicate entries found: ", len(bruteForcePairs),"\n")
print("Brute force approach took ", BFtime / execTime, " times the time taken by the LSH approach")

#print("Indexes of duplicate pairs: \n")
#for p in bruteForcePairs:
#    print(int(p[0]), int(p[1]))

The brute force code took  425.94332287293537 ms to detect duplicates

Total duplicate entries found:  25 

Brute force approach took  37.67352928961274  times the time taken by the LSH approach
